# Imports

In [3]:
import json


import ccxt
import requests


import logging

import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

# %load_ext autoreload
# %autoreload 2


In [4]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmampTask4631_Create_random_workload_for_broker_run'
  hash='75ec043c2'
  # Last commits:
    * 75ec043c2 vlady    CmampTask4631: continue to play around with the random orders parameters (   7 hours ago) Mon Jul 3 06:31:13 2023  (HEAD -> CmampTask4631_Create_random_workload_for_broker_run, origin/CmampTask4631_Create_random_workload_for_broker_run)
    * 6ce45a105 vlady    CmampTask4631: play around with the order size                    (  10 hours ago) Mon Jul 3 03:40:22 2023           
    * fca22b11c vlady    CmampTask4631: playing around max_order_notional                 

# Get last prices from coinmarketcap.com

In [103]:
CMC_PRO_API_KEY = "31eb170c-a347-4c3e-824a-76c5aaa8b22d"
LIMIT = "1000"
latest_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
parameters = {
  'start':'1',
  'limit': LIMIT,
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': CMC_PRO_API_KEY,
}

In [104]:
result = requests.get(
    latest_url,
    params=parameters,
    headers=headers
)

In [105]:
mode = "trade"
vendor = "ccxt"
version = "v7.1"
universe = ivcu.get_vendor_universe(
    vendor, mode, version=version
)


In [106]:
market_data = pd.DataFrame(result.json()["data"])

In [107]:
market_data = market_data.groupby('symbol').apply(lambda x: x[x['id'] == x['id'].min()])

In [108]:
market_data

,,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote
symbol,,,,,,,,,,,,,,,,,,,
$MONG,726,24793,MongCoin,$MONG,mongcoin,18,2023-04-28T06:50:35.000Z,[memes],6.900000e+14,5.811960e+14,6.900000e+14,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",727,5.686374e+14,1.220004e+07,NaN,2023-07-03T14:06:00.000Z,"{'USD': {'price': 2.1454864269822927e-08, 'vol..."
1INCH,101,8104,1inch Network,1INCH,1inch,347,2020-12-25T00:00:00.000Z,"[decentralized-exchange-dex-token, defi, walle...",NaN,9.433451e+08,1.500000e+09,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",102,9.433451e+08,3.173484e+08,NaN,2023-07-03T14:06:00.000Z,"{'USD': {'price': 0.336407506126416, 'volume_2..."
1WO,999,2601,1World,1WO,1world,3,2018-03-21T00:00:00.000Z,[algorand-ecosystem],NaN,3.721945e+07,3.721945e+07,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",1000,NaN,NaN,NaN,2023-07-03T14:06:00.000Z,"{'USD': {'price': 0.11197148473683413, 'volume..."
AAVE,40,7278,Aave,AAVE,aave,615,2020-10-02T00:00:00.000Z,"[defi, dao, yield-farming, three-arrows-capita...",1.600000e+07,1.445839e+07,1.600000e+07,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",41,NaN,NaN,0.495452,2023-07-03T14:06:00.000Z,"{'USD': {'price': 69.87740555321332, 'volume_2..."
ABBC,198,3437,ABBC Coin,ABBC,abbc-coin,78,2018-10-12T00:00:00.000Z,"[dpos, platform, payments]",1.500000e+09,1.421562e+09,1.500000e+09,False,None,199,NaN,NaN,NaN,2023-07-03T14:06:00.000Z,"{'USD': {'price': 0.06996331385129455, 'volume..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZKS,798,8202,ZKSpace,ZKS,zkswap,42,2021-01-07T00:00:00.000Z,"[decentralized-exchange-dex-token, zero-knowle...",1.000000e+09,1.974400e+08,1.000000e+09,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",799,NaN,NaN,NaN,2023-07-03T14:06:00.000Z,"{'USD': {'price': 0.04573822105238773, 'volume..."
ZRX,149,1896,0x Protocol,ZRX,0x,283,2017-08-16T00:00:00.000Z,"[platform, decentralized-exchange-dex-token, d...",1.000000e+09,8.474961e+08,1.000000e+09,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",150,NaN,NaN,NaN,2023-07-03T14:06:00.000Z,"{'USD': {'price': 0.21648834567804126, 'volume..."
ZYN,922,4951,Zynecoin,ZYN,zynecoin,3,2020-07-07T00:00:00.000Z,[mineable],NaN,3.271319e+07,9.601725e+07,False,None,923,NaN,NaN,NaN,2023-07-03T14:06:00.000Z,"{'USD': {'price': 0.18174421903657606, 'volume..."


In [109]:
market_prices = market_data[["symbol", "quote"]]

In [110]:
market_prices["price"] = market_prices["quote"].apply(lambda x: x["USD"]["price"])

In [111]:
market_prices.drop(columns=["quote"], inplace=True)

In [112]:
market_prices

,,symbol,price
symbol,,,
$MONG,726,$MONG,2.145486e-08
1INCH,101,1INCH,3.364075e-01
1WO,999,1WO,1.119715e-01
AAVE,40,AAVE,6.987741e+01
ABBC,198,ABBC,6.996331e-02
...,...,...,...
ZKS,798,ZKS,4.573822e-02
ZRX,149,ZRX,2.164883e-01
ZYN,922,ZYN,1.817442e-01


In [113]:
universe_symbols = pd.Series([
    symbol.split("_")[0]
    for symbol in universe["binance"]
])

In [114]:
universe_symbols[~universe_symbols.isin(market_prices.symbol)]

Series([], dtype: object)

In [115]:
market_prices = market_prices[market_prices.symbol.isin(universe_symbols)]

In [116]:
market_prices = market_prices.set_index("symbol")

In [121]:
prices = market_prices.to_dict(orient="index")

In [122]:
prices = {
    key: prices[key]["price"]
    for key in prices
}

In [123]:
prices

{'APE': 2.197905043658875,
 'AVAX': 13.162287133563312,
 'AXS': 6.261364820219348,
 'BAKE': 0.11743721712923332,
 'BNB': 248.73373185917208,
 'BTC': 30685.3848801616,
 'CRV': 0.7729292526995412,
 'CTK': 0.64635351967252,
 'DOGE': 0.06748289718347907,
 'DOT': 5.424402039243173,
 'DYDX': 2.086527472817396,
 'ETH': 1964.13267996661,
 'FTM': 0.31823110515831043,
 'GMT': 0.10909453342241375,
 'LINK': 6.607307622727324,
 'MATIC': 0.6916823433191273,
 'NEAR': 1.4524186289010763,
 'OGN': 0.0848774014247122,
 'RUNE': 1.0863405284306937,
 'SAND': 0.44480107827371335,
 'SOL': 19.152519190599996,
 'STORJ': 0.3475357284032491,
 'UNFI': 3.5007835966573153,
 'WAVES': 1.9205050905298902,
 'XRP': 0.48170087502151954}